<a href="https://colab.research.google.com/github/JennaBeutler/APRD-6342/blob/master/Exercise_1_Basic_Ad_Analyses.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python Exercise 1: Basic Ad Analyses  
### APRD 6342  
### Fall 2019  
### Jenna Beutler 

First importing all necessary packages and mounting the drive to Colab

In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import calendar
import statsmodels.api as sm

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Loading the data into colab and looking at the head of the data 

In [2]:
 path = '/content/drive/My Drive/Colab Notebooks/Python Exercises/Travel Pony Facebook.csv'
df = pd.read_csv(path)
df.head()

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD)
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.00,1.219512
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.00,1.395349
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.00,1.190476
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.00,1.167315
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.59,2.512857




---



#### Question 1: Create a new column 'Cost per Impression'. In terms of this metric, what day of the week works best?  
First, I create a column that divides Amount Spent by Impressions  
Then create a weekday column


In [3]:
df['cost per impression'] = df['Amount Spent (USD)'] / df['Impressions']

#Convert Start Date column to a datetime object
df['Start Date'] = pd.to_datetime(df['Start Date'])

# Create a new column called 'Day of Week' and populate it using the date column
df['Day of Week'] = df['Start Date'].dt.day_name()
df.head()

,Start Date,End Date,Account,Campaign Name,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD),cost per impression,Day of Week
0,2013-12-02,2013-12-02,Bob Jackson,Cyber Weekend,278117,2.314303,643647,336,313,300.00,3,246,243,0,0,0,0,243,100.00,1.219512,0.000466,Monday
1,2013-12-01,2013-12-01,Bob Jackson,Cyber Weekend,221255,2.079090,460009,376,348,300.00,6,215,209,0,0,0,0,209,50.00,1.395349,0.000652,Sunday
2,2013-11-30,2013-11-30,Bob Jackson,Cyber Weekend,267915,2.499069,669538,383,345,300.00,2,252,250,0,0,0,0,250,150.00,1.190476,0.000448,Saturday
3,2013-11-29,2013-11-29,Bob Jackson,Cyber Weekend,280332,2.365406,663099,301,275,300.00,3,257,254,0,0,0,0,254,100.00,1.167315,0.000452,Friday
4,2013-11-22,2013-11-22,Bob Jackson,TravelPony: General Ads,14269,1.000420,14275,64,61,35.18,2,14,12,0,0,0,0,12,17.59,2.512857,0.002464,Friday


Then I create a pivoted version of the data, grouping by day of the week, so I can see what days performed best. 

In [0]:
pivotday = df.pivot_table(index = 'Day of Week', aggfunc = 'mean')

# Sort the values by cost per impression to see which day of the week performs best and worst. 
pivotday.sort_values(by= ['cost per impression'])

,Amount Spent (USD),Clicks,Cost per Page Engagement (USD),Cost per Page Like (USD),Frequency,Impressions,Page Engagement,Page Likes,Photo Views,Post Comments,Post Engagement,Post Likes,Post Shares,Reach,Unique Clicks,Website Clicks,cost per impression
Day of Week,,,,,,,,,,,,,,,,,
Saturday,9.625511,10.014028,0.415395,2.650510,1.014517,3152.184369,9.573146,0.923848,3.595190,0.048096,8.649299,1.102204,0.212425,2281.755511,8.857715,3.691383,0.002629
Monday,7.427097,7.536503,0.492924,2.273947,1.017603,2525.074703,7.634975,0.670628,2.994907,0.040747,6.964346,1.095076,0.225806,1842.864177,6.521222,2.607810,0.002910
Wednesday,7.956024,7.781726,0.539237,2.067530,1.009059,1516.478849,8.131980,0.934010,3.225042,0.067682,7.197970,1.241963,0.187817,1494.235195,6.524535,2.475465,0.002981
Tuesday,9.455084,9.419831,0.592907,2.510189,1.012193,1926.738397,9.679325,1.094937,3.877637,0.065401,8.584388,1.280591,0.202532,1889.215190,8.151899,3.158228,0.003040
Thursday,7.640209,8.054007,0.560598,2.230755,1.011668,1449.837979,8.634146,0.855401,4.012195,0.094077,7.778746,1.371080,0.200348,1413.081882,6.980836,2.101045,0.003398
Sunday,9.676534,9.981092,0.491080,2.475630,1.016073,2775.644958,8.794118,0.831933,3.220588,0.029412,7.960084,0.962185,0.218487,2203.596639,8.808824,3.529412,0.003688
Friday,10.014741,10.147410,0.476606,2.475714,1.012024,3037.984064,10.635458,0.844622,4.745020,0.051793,9.790837,1.424303,0.250996,2238.474104,8.848606,3.318725,0.004097




---



#### Question 2: Generate the correlation values between amount spent and the following:  
- reach  
- frequency  
- unique clicks  
- page likes

In [0]:
#Create a correlation matrix, pick out the numbers I need to answer the question. 
df.corr()

,Reach,Frequency,Impressions,Clicks,Unique Clicks,Amount Spent (USD),Page Likes,Page Engagement,Post Engagement,Post Likes,Post Comments,Post Shares,Photo Views,Website Clicks,Cost per Page Like (USD),Cost per Page Engagement (USD),cost per impression
Reach,1.000000,0.334101,0.966851,0.709466,0.722249,0.703124,0.304388,0.597648,0.605244,0.151727,0.100254,0.153288,0.205678,0.858860,0.646366,0.168697,0.028005
Frequency,0.334101,1.000000,0.386949,0.128473,0.135103,0.130201,0.000182,0.111645,0.119780,-0.015511,-0.008028,-0.012113,-0.009054,0.240826,0.154855,0.043232,-0.018217
Impressions,0.966851,0.386949,1.000000,0.538032,0.553394,0.532913,0.158246,0.459510,0.474330,0.065275,0.043358,0.067018,0.090544,0.769145,0.565503,0.091639,0.006973
Clicks,0.709466,0.128473,0.538032,1.000000,0.998287,0.883411,0.591499,0.797945,0.786167,0.465285,0.242256,0.407737,0.448348,0.817139,0.594057,0.209975,0.088315
Unique Clicks,0.722249,0.135103,0.553394,0.998287,1.000000,0.882993,0.584614,0.787337,0.775598,0.445915,0.224601,0.379449,0.432454,0.822962,0.597671,0.213844,0.086798
Amount Spent (USD),0.703124,0.130201,0.532913,0.883411,0.882993,1.000000,0.757612,0.819253,0.789348,0.386383,0.259781,0.420018,0.479713,0.817490,0.637906,0.323046,0.119201
Page Likes,0.304388,0.000182,0.158246,0.591499,0.584614,0.757612,1.000000,0.650871,0.579933,0.454491,0.290983,0.432091,0.402059,0.477790,0.304196,0.271398,0.143931
Page Engagement,0.597648,0.111645,0.459510,0.797945,0.787337,0.819253,0.650871,1.000000,0.995945,0.700280,0.387753,0.648593,0.814885,0.731632,0.617181,0.147983,0.117141
Post Engagement,0.605244,0.119780,0.474330,0.786167,0.775598,0.789348,0.579933,0.995945,1.000000,0.697600,0.381613,0.644780,0.826786,0.728476,0.626231,0.126639,0.108649
Post Likes,0.151727,-0.015511,0.065275,0.465285,0.445915,0.386383,0.454491,0.700280,0.697600,1.000000,0.424246,0.689834,0.639658,0.240632,0.267482,0.058336,0.106425




---



#### Question 3: Finally, perform a simple multiple regression analysis where Unique Clicks is the dependent variable and Reach and Frequency are the independent (predictor) variables.

In [0]:
# Setting up the variables to run a regression 
X = df[['Reach', 'Frequency']]
y = df['Unique Clicks']

# saving a fitted OLS regression and its predictions
model = sm.OLS(y, X).fit()
predictions = model.predict(X)

#looking at the summary of the regression
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          Unique Clicks   R-squared (uncentered):                   0.557
Model:                            OLS   Adj. R-squared (uncentered):              0.556
Method:                 Least Squares   F-statistic:                              2325.
Date:                Sat, 09 Nov 2019   Prob (F-statistic):                        0.00
Time:                        18:38:23   Log-Likelihood:                         -15973.
No. Observations:                3705   AIC:                                  3.195e+04
Df Residuals:                    3703   BIC:                                  3.196e+04
Df Model:                           2                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Reach          0.0019   3.12e-05     62.490      0.000       0.002       0.002
Frequency      3.6139      0.298     12.109      0.000       3.029       4.199
==============================================================================
Omnibus:                     5107.616   Durbin-Watson:                   0.803
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          5128392.747
Skew:                           7.331   Prob(JB):                         0.00
Kurtosis:                     184.674   Cond. No.                     9.84e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.84e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""